In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import numpy as np
from datetime import datetime
import os
import glob
import shutil
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import yfinance as yf


In [2]:
# Appends new row to existing csv file
# path = "TEST/testing.csv"

def append_row_toCSV(path, toadd_data):
    mydata = pd.read_csv(path)
    newdata = pd.DataFrame([toadd_data], columns=mydata.columns)
    mydata = pd.concat([mydata, newdata], ignore_index=True)
    mydata.to_csv(path, index=False)



In [ ]:
Earnings beat/miss

Analyst ratings compared to current price
https://www.marketbeat.com/stocks/NASDAQ/AAPL/price-target/

In [151]:
# finds earnings estimate and and EPS 
def add_EPS_estimate_reported(url):
    html_text = requests.get(url).text
    soup = BeautifulSoup(html_text, "html.parser")
    body = soup.find("table", class_="scroll-table sort-table", id="earnings-history")

    body = body.find("tbody").find_all("tr")
    body.pop(0)
    mylist = []
    for i in body:
        # print(i.find("td").contents)
        while True:
            date_EPS = i.find("td")
            consensus_EPS = date_EPS.find_next("td").find_next("td") 
            reported_EPS = consensus_EPS.find_next("td")
            if str(date_EPS.contents[0])[0] == "<":
                break
            date_EPS = date_EPS.contents
            consensus_EPS = consensus_EPS.contents
            reported_EPS = reported_EPS.contents
            mylist2 = [date_EPS,consensus_EPS,reported_EPS]
            mylist.append(mylist2)
            break
    return mylist

In [3]:
#check if the difference betweens two dates are below 5 to match with their respective quarterly report dates
#checks if the data on consensus/reported EPS match the quarterly report one
# since it is possible for them to be different as they could be released at different dates and depends on whether the websites I used
# to scrape from update it instantenously or not, thus ±5 days error range.
def check_day_diff(mydatestr, quarterURL):
    
    date_1 = datetime.strptime(mydatestr, f'%m/%d/%Y')

    myurl = pd.read_csv(quarterURL)
    date_string = myurl.loc[1][0]
    date_2 = datetime.strptime(date_string, f'%Y-%m-%d %H:%M:%S')

    day_difference = abs((date_2 - date_1).days)
    if day_difference < 5:
        return True
    else:
        return False

In [264]:
# sorts directory into a list, then add consensus and reported EPS to each stock's csv files
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
        all_files.sort(reverse=True) #gives a SORTED list of all .csv paths of that particular stock 
        url = 'https://www.marketbeat.com/stocks/NASDAQ/'+"MSFT"+'/earnings/'
        eps_list = add_EPS_estimate_reported(url)
        print(eps_list)
        i=0
        try:
            for quarterURL in all_files: #loops around each .csv path
                date = ''.join(eps_list[i][0])
                mybool = check_day_diff(date, quarterURL) #checks if the data on consensus/reported EPS match the quarterly report one
                consensus_EPS = ''.join(eps_list[i][1]).replace("$","") #gets consensus data, turning it into a string in the format i.e. $0.48 then getting rid of the $ sign --> to get 0.48
                reported_EPS = ''.join(eps_list[i][2]).replace("$","")
                
                percentage = (float(reported_EPS)/float(consensus_EPS))-1
                print(percentage)
                
                if mybool == True:
                    path = "TEST/testing.csv"
                    #path = quarterURL
                    append_row_toCSV(path, percentage) #adds percentage of surprise to csv file
                i+=1
        except:
            pass
        break
    break


[[['10/25/2022'], ['$2.29'], ['$2.35']], [['7/26/2022'], ['$2.28'], ['$2.23']], [['4/26/2022'], ['$2.18'], ['$2.22']], [['1/25/2022'], ['$2.29'], ['$2.48']], [['10/25/2021'], ['$2.08'], ['$2.27']], [['7/26/2021'], ['$1.92'], ['$2.17']], [['4/27/2021'], ['$1.76'], ['$1.95']], [['1/25/2021'], ['$1.64'], ['$2.03']], [['10/27/2020'], ['$1.53'], ['$1.82']], [['7/22/2020'], ['$1.34'], ['$1.46']], [['4/29/2020'], ['$1.27'], ['$1.40']], [['1/29/2020'], ['$1.32'], ['$1.51']], [['10/23/2019'], ['$1.24'], ['$1.38']], [['7/18/2019'], ['$1.21'], ['$1.37']], [['4/24/2019'], ['$1.00'], ['$1.1375']], [['1/30/2019'], ['$1.09'], ['$1.10']], [['10/24/2018'], ['$0.96'], ['$1.14']], [['7/19/2018'], ['$1.08'], ['$1.13']], [['4/26/2018'], ['$0.85'], ['$0.95']], [['1/31/2018'], ['$0.86'], ['$0.96']], [['10/26/2017'], ['$0.72'], ['$0.84']], [['7/20/2017'], ['$0.71'], ['$0.98']], [['4/27/2017'], ['$0.69'], ['$0.73']], [['1/26/2017'], ['$0.79'], ['$0.83']], [['10/20/2016'], ['$0.68'], ['$0.76']], [['7/19/2016'],

Used yfinance instead of scraping it myself since the combination of url of each stock on marketbeat is different
i.e. 
NASDAQ/MSFT/earnings
NYSE/JPM/earnings

also yfinance has more historical data (see below compared to above), both MSFT


In [6]:
import yfinance as yf


msft = yf.Ticker("msft")
earnings_list = msft.get_earnings_dates(limit=200)
earnings_list.reset_index(inplace=True)
earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
eps_list = earnings_list.reset_index(drop=True)

eps_list[0:50]

,Earnings Date,EPS Estimate,Reported EPS,Surprise(%)
0,2022-10-25 12:00:00-04:00,2.30,2.35,0.0204
1,2022-07-26 12:00:00-04:00,2.29,2.23,-0.0275
2,2022-04-26 12:00:00-04:00,2.19,2.22,0.0160
3,2022-01-25 11:00:00-05:00,2.31,2.48,0.0727
4,2021-10-26 12:00:00-04:00,2.08,2.27,0.0940
5,2021-07-27 12:00:00-04:00,1.92,2.17,0.1326
6,2021-04-27 12:00:00-04:00,1.78,2.03,0.1411
7,2021-01-26 11:00:00-05:00,1.64,2.03,0.2363
8,2020-10-27 12:00:00-04:00,1.54,1.82,0.1788
9,2020-07-22 12:00:00-04:00,1.34,1.46,0.0871


In [3]:
#Adjusted for Yfinance
#check if the difference betweens two dates are below 6 to match with their respective quarterly report dates
def check_day_diff(date_1,QuarterURL):
    myurl = pd.read_csv(QuarterURL)
    date_string = myurl.loc[1][0]
    date_2 = datetime.strptime(date_string, f'%Y-%m-%d %H:%M:%S')
    
    day_difference = abs((date_2 - date_1).days)
    if day_difference < 5:
        return True
    else:

        return False



In [5]:
# sorts directory into a list, then add the difference between consensus and reported EPS to each stock's csv files
def get_EPS(stock, sector):
    all_files = glob.glob('sectors/'+str(sector)+'/'+str(stock)+'/'+str(stock)+'20*[0-9]**[0-9]*[1-4].*')
    all_files.sort(reverse=True) #gives a SORTED list of all .csv paths of that particular stock 
    try:
        yf_stock = yf.Ticker(stock)
        earnings_list = yf_stock.get_earnings_dates(limit=200) #uses yfinance and gets historical eps consensus/reported data
        earnings_list.reset_index(inplace=True)
        earnings_list = earnings_list[earnings_list['Surprise(%)'].notna()]
        eps_list = earnings_list.reset_index(drop=True)
        number_csv = 0
        number_EPS = 0
        number_csv_2 = 0
        try:
            while number_csv < len(all_files): #loops around each .csv path
                quarterURL = all_files[number_csv] 
                percentage_surprise = float(eps_list['Surprise(%)'][number_EPS])
                date = eps_list['Earnings Date'][number_EPS]
                date = date.replace(tzinfo=None) #fixes "Cannot subtract tz-naive and tz-aware datetime-like objects" error
                mybool = check_day_diff(date, quarterURL) #checks if the data on consensus/reported EPS match the quarterly report one
                
                read_csv = pd.read_csv(quarterURL)

                if mybool == True:
                    number_csv += 1
                    number_EPS += 1
                    number_csv_2 += 1
                    if len(read_csv) > 3:
                        read_csv = read_csv[:3] # if data has failed to load (i.e. printed as NaN), the previous Nan Data will be wiped, and the process will be redone
                    newdata = pd.DataFrame([percentage_surprise], columns=read_csv.columns)
                    read_csv = pd.concat([read_csv, newdata], ignore_index=True)
                    read_csv.to_csv(quarterURL, index=False)
                    return True
# number_csv_2 is used with another mybool2 to check if the EPS quarterly report date matches with the date on the .csv file, in case it was reversed
#e.g.
# case 1: uses mybool
# csv       eps
# 2020      2022 (mybool = False)
# 2020      2021 (mybool = False)
# 2020      2020 (MATCH, mybool = True)
# 
# case 2: uses mybool2
# csv       eps
# 2022      2020 (mybool = False)
# 2021      2020 (mybool = False)
# 2020      2020 (MATCH, mybool2 = True)
#

                elif mybool == False:
                    number_EPS += 1
                    number_csv_2 += 1
                    quarterURL = all_files[number_csv_2] 
                    percentage_surprise = float(eps_list['Surprise(%)'][number_EPS])/100
                    date = eps_list['Earnings Date'][number_EPS]
                    date = date.replace(tzinfo=None) 
                    mybool2 = check_day_diff(date, quarterURL)
                    if mybool2 == True:
                        number_EPS -=1
                        read_csv = pd.read_csv(quarterURL)
                        number_csv += 1
                        number_EPS += 1
                        number_csv_2 += 1

                    if len(read_csv) < 4:
                        append_row_toCSV(quarterURL, np.NaN)
                
        except:
            pass        
    except:
        print("ERROR", stock, "NOT FOUND!!!!!!") 
        with open("not_found_list_stocks.txt", "a") as file_object: #writes to file a list of stocks' EPS that couldn't be found on yfinance
            file_object.write(sector)
            file_object.write("\n")
            file_object.write(stock)
            file_object.write("\n")
            return False
        pass
    


In [ ]:
#Adjusted for Yfinance
sectorlist = ["automobiles", "banks", "capital-goods", "commercial-services", 
"consumer-durables", "consumer-retailing", "consumer-services", "diversified-financials",
"energy", "food-beverage-tobacco", "healthcare", "household", "insurance", "materials", "media", 
"pharmaceuticals-biotech", "real-estate", "retail", "semiconductors", "software",
"tech", "telecom", "transportation", "utilities"]

for sector in sectorlist:
    print(sector, "sector start ................................................................")
    filelist = os.listdir("sectors/"+sector)
    try:
        filelist.remove('.DS_Store')
    except:
        pass
    for stock in filelist:
        print(stock, "start")

        boolean = get_EPS(stock, sector) #calls get_EPS function and notes EPS info on the relevant .csv files
        print(boolean)

I noticed that some EPS info of certain stocks were termporarily uncallable using the yfinance API.
So I gathered a list of all stocks with the error onto a text file:
"CAT: No earnings dates found, symbol may be delisted"
The EPS data is then re-collected by looping over these stocks.

In [5]:
with open("not_found_list_stocks.txt", "r") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    mylist = f.read().splitlines()

print(mylist)

for i in range(0, len(mylist), 2):
    sector = mylist[i]
    stock = mylist[i+1]
    boolean = get_EPS(stock, sector)
    print(boolean)
    if boolean == True:
        print(stock)
        mylist[i] = ""
        mylist[i+1] = ""

mylist = list(filter(None, mylist)) # deletes all empty strings (i.e. stocks that have been replaced from the text "not found list text file"

mystr = '\n'.join(mylist)

with open("not_found_list_stocks.txt", "w") as f: #writes to file a list of stocks' EPS that couldn't be found on yfinance
    f.write(mystr)

['automobiles', 'TEN', 'automobiles', 'XL', 'banks', 'ABTX', 'banks', 'FBC', 'banks', 'HBAN', 'banks', 'WSBC', 'banks', 'TBK', 'banks', 'COLB', 'banks', 'PFS', 'banks', 'FFBC', 'banks', 'NTB', 'banks', 'FULT', 'banks', 'AGM', 'banks', 'CVBF', 'banks', 'HTH', 'banks', 'CMA', 'banks', 'TCBI', 'banks', 'COOP', 'banks', 'VLY', 'banks', 'ONB', 'banks', 'SI', 'banks', 'SFBS', 'banks', 'UCBI', 'banks', 'AUB', 'banks', 'SBNY', 'banks', 'WBS', 'banks', 'SFNC', 'capital-goods', 'FBHS', 'capital-goods', 'NPO', 'capital-goods', 'GTLS', 'capital-goods', 'STEM', 'capital-goods', 'AIN', 'capital-goods', 'ST', 'capital-goods', 'CMI', 'commercial-services', 'MANT', 'commercial-services', 'NLSN', 'consumer-services', 'HLG', 'consumer-services', 'TMX', 'consumer-services', 'STON', 'energy', 'CLR', 'food-beverage-tobacco', 'LNDC', 'healthcare', 'HNGR', 'healthcare', 'CHNG', 'healthcare', 'CVET', 'household', 'VERU', 'materials', 'GCP', 'materials', 'TMST', 'pharmaceuticals-biotech', 'GBT', 'pharmaceutical

In [ ]:
"""
automobiles
TEN
automobiles
XL
banks
ABTX
banks
FBC
banks
HBAN
banks
WSBC
banks
TBK
banks
COLB
banks
PFS
banks
FFBC
banks
NTB
banks
FULT
banks
AGM
banks
CVBF
banks
HTH
banks
CMA
banks
TCBI
banks
COOP
banks
VLY
banks
ONB
banks
SI
banks
SFBS
banks
UCBI
banks
AUB
banks
SBNY
banks
WBS
banks
SFNC
capital-goods
FBHS
capital-goods
NPO
capital-goods
GTLS
capital-goods
STEM
capital-goods
AIN
capital-goods
ST
capital-goods
CMI
commercial-services
MANT
commercial-services
NLSN
consumer-services
HLG
consumer-services
TMX
consumer-services
STON
energy
CLR
food-beverage-tobacco
LNDC
healthcare
HNGR
healthcare
CHNG
healthcare
CVET
household
VERU
materials
GCP
materials
TMST
pharmaceuticals-biotech
GBT
pharmaceuticals-biotech
CCXI
pharmaceuticals-biotech
AGIO
pharmaceuticals-biotech
PCRX
pharmaceuticals-biotech
ISEE
pharmaceuticals-biotech
TECH
pharmaceuticals-biotech
MYOV
pharmaceuticals-biotech
RPRX
real-estate
BRG
real-estate
CTT
real-estate
APTS
real-estate
ADC
real-estate
AHH
real-estate
MMI
real-estate
COLD
real-estate
GOOD
real-estate
RLJ
real-estate
OPI
real-estate
SITC
real-estate
HT
real-estate
UDR
real-estate
UMH
real-estate
JLL
real-estate
HIW
retail
LL
retail
BBWI
retail
SCVL
retail
LE
retail
DBI
retail
TTSH
retail
BBY
retail
DKS
retail
BBW
retail
FL
retail
ARHS
retail
TGT
retail
VIPS
retail
ZUMZ
retail
GES
retail
BBBY
retail
AZO
retail
MUSA
retail
AN
retail
GPC
retail
RCII
retail
JMIA
retail
SAH
semiconductors
HIMX
semiconductors
AMD
semiconductors
WOLF
semiconductors
LRCX
semiconductors
PI
semiconductors
TER
semiconductors
TGAN
semiconductors
DQ
semiconductors
SITM
semiconductors
GSIT
semiconductors
ACMR
semiconductors
ASYS
semiconductors
CYBE
semiconductors
SQNS
semiconductors
SIMO
semiconductors
ALGM
software
SQ
software
PING
software
AVLR
software
SWCH
software
CWAN
software
IT
software
ZEN
software
ALRM
software
NOW
software
DV
software
KNBE
software
PAYC
software
VRNT
software
NCR
software
AFRM
software
NTNX
software
FTNT
software
MANH
software
PAYX
software
INTU
software
OKTA
software
FLT
software
FLYW
software
CHKP
software
ALTR
software
DOX
software
DLB
software
MQ
software
CTXS
tech
MOVE
tech
OSIS
tech
MICT
tech
BMI
tech
ITI
tech
IPGP
tech
LITE
tech
CASA
tech
QMCO
tech
NSSC
tech
HPQ
tech
OUST
tech
FFIV
tech
DELL
tech
CNXN
tech
DAKT
tech
ALOT
tech
NATI
tech
GNSS
tech
VSAT
tech
RFIL
tech
DGII
tech
TTMI
tech
KODK
tech
VNT
tech
AKTS
tech
AIRG
tech
TACT
tech
SSYS
tech
TDY
tech
PAR
tech
SANM
transportation
SB
transportation
ULCC
transportation
SAIA
transportation
PANL
transportation
AAL
transportation
UBER
transportation
RLGT
transportation
GNK
transportation
CAR
transportation
ALK
transportation
KEX
transportation
BEST
transportation
AAWW
transportation
ALGT
transportation
GLG
transportation
GRIN
transportation
CHRW
transportation
LSTR
transportation
JBHT
transportation
ZIM
transportation
USX
transportation
NETI
transportation
KNX
transportation
UP
transportation
CSX
transportation
NM
transportation
GSL
transportation
CMRE
transportation
HA
transportation
SNDR
transportation
GLG
transportation
GRIN
transportation
CHRW
transportation
LSTR
transportation
JBHT
transportation
ZIM
transportation
USX
transportation
NETI
transportation
KNX
transportation
UP
transportation
CSX
transportation
NM
transportation
GSL
transportation
CMRE
transportation
HA
transportation
SNDR
transportation
YMM
transportation
UAL
transportation
HUBG
transportation
ZTO
transportation
EGLE
transportation
ODFL
transportation
MESA
transportation
FDX
telecom
PTNR
telecom
GOGO
telecom
LILA
utilities
NWN
utilities
PEG
utilities
OGE
utilities
AZRE
utilities
SO
utilities
DTE
utilities
IDA
utilities
GWRS
utilities
XEL
utilities
SR
utilities
CMS
utilities
NWE
utilities
EVRG
utilities
NEE
utilities
CWCO
utilities
CPK
utilities
RGCO
utilities
PCYO
utilities
NJR
utilities
UGI
utilities
BKH
utilities
EXC
utilities
ES
"""

Before (above) compared to after (below)

In [ ]:
"""
automobiles
TEN
automobiles
XL
banks
ABTX
banks
FBC
banks
TBK
capital-goods
FBHS
commercial-services
MANT
commercial-services
NLSN
consumer-services
HLG
consumer-services
TMX
consumer-services
STON
energy
CLR
food-beverage-tobacco
LNDC
healthcare
HNGR
healthcare
CHNG
healthcare
CVET
materials
GCP
pharmaceuticals-biotech
GBT
pharmaceuticals-biotech
CCXI
real-estate
BRG
real-estate
CTT
real-estate
APTS
semiconductors
CYBE
software
PING
software
AVLR
software
SWCH
software
ZEN
software
CTXS
tech
MOVE
transportation
GLG
transportation
GLG"""